<a href="https://colab.research.google.com/github/sundarp17/Housing/blob/main/Housing_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors

city = pd.read_csv(r'/content/cities.csv')
city_census = pd.read_csv(r'/content/census_cities.csv')
city.head()

,ID,city,state,state_code,code,time_created
0,1,Alabaster city,Alabama,1,820,2021-01-19 14:47:03
1,2,Albertville city,Alabama,1,988,2021-01-19 14:47:03
2,3,Anniston city,Alabama,1,1852,2021-01-19 14:47:03
3,4,Athens city,Alabama,1,2956,2021-01-19 14:47:03
4,5,Auburn city,Alabama,1,3076,2021-01-19 14:47:03


In [2]:
city.isnull().sum()

ID              0
city            0
state           0
state_code      0
code            0
time_created    0
dtype: int64

In [3]:
city_census.head()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year,Column1.2.2.2.135
0,4026,1608,35.8,33576,38782,61904.0,4394.0,321.0,2502.0,99.0,915.0,78288.0,29102.0,56808.0,4564.0,5569.0,5244.0,6888.0,3839.0,1809.0,1189.0,8868.0,2680.0,5911.0,277.0,60917.0,9832.0,38103.0,6261.0,21407.0,9527.0,908.0,39388.0,33460.0,1154.0,1285.0,69543,44772,24771,30865,29102,1763,30865.0,71931.0,59910.0,7309.0,2672.0,1492.0,980.0,2616.0,12230.0,10073.0,10970.0,5610.0,72358.0,68312.0,52445.0,15374.0,493.0,2016,NaN
1,4027,1609,37.8,19178,19694,31181.0,185.0,238.0,5405.0,0.0,221.0,53878.0,16312.0,41226.0,2974.0,5028.0,3068.0,3772.0,793.0,254.0,423.0,8595.0,3318.0,4650.0,627.0,28858.0,5616.0,16982.0,4137.0,10951.0,1749.0,145.0,17514.0,14636.0,735.0,532.0,37885,23368,14517,17679,16312,1367,17679.0,38442.0,32713.0,3804.0,1506.0,419.0,1049.0,1449.0,8830.0,4676.0,4528.0,1922.0,38872.0,36297.0,26777.0,9300.0,220.0,2016,NaN
2,4028,1610,37.9,23856,24084,41235.0,1555.0,324.0,2391.0,0.0,430.0,102471.0,19604.0,79384.0,2059.0,2362.0,3236.0,4377.0,3559.0,1981.0,2030.0,2000.0,0.0,1244.0,756.0,45384.0,6217.0,24137.0,3326.0,16261.0,4073.0,477.0,25370.0,21049.0,762.0,1233.0,47227,34438,12789,20290,19604,686,20290.0,46960.0,40526.0,3261.0,1316.0,1491.0,87.0,623.0,4898.0,5210.0,11501.0,8611.0,47940.0,44959.0,35093.0,9629.0,237.0,2016,NaN
3,4029,1611,38.5,29889,30197,48892.0,3334.0,971.0,1395.0,0.0,3252.0,66069.0,28055.0,49243.0,5448.0,5515.0,5701.0,7779.0,2786.0,446.0,380.0,7711.0,1635.0,4954.0,1122.0,51743.0,6924.0,31057.0,3942.0,18947.0,6971.0,1197.0,31637.0,26747.0,864.0,580.0,59256,34132,25124,29761,28055,1706,29761.0,59617.0,53169.0,4905.0,845.0,297.0,1109.0,2228.0,15992.0,9769.0,7945.0,3063.0,60086.0,56511.0,48312.0,7344.0,855.0,2016,NaN
4,4030,1612,40.3,15215,16486,29207.0,472.0,0.0,341.0,0.0,854.0,71615.0,13259.0,59445.0,1611.0,2326.0,2722.0,3459.0,1835.0,995.0,311.0,1920.0,410.0,897.0,613.0,29486.0,5907.0,15304.0,3581.0,5912.0,5088.0,723.0,15898.0,13366.0,130.0,594.0,31385,22033,9352,14128,13259,869,14128.0,31448.0,27670.0,2150.0,1032.0,596.0,662.0,667.0,6730.0,4977.0,4718.0,1563.0,31701.0,30759.0,26656.0,3988.0,115.0,2016,NaN


In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
city_census = city_census.drop('Column1.2.2.2.135',axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [5]:
city_census['year'].unique()

array([2016, 2017, 2018, 2019])

In [6]:
city_census.isnull().sum()

Census_ID                                   0   
city_id                                     0   
median_age                                  0   
male                                        0   
female                                      0   
white_alone                                 1915
black_africa                                1915
native                                      1915
asian                                       1915
islander                                    1915
other                                       1915
median_family_income                        2   
household_income                            4   
median_household_income                     1   
household_less_20                           4   
household_20_39                             4   
household_40_59                             4   
household_60_99                             4   
household_100_149                           4   
household_150_200                           4   
household_200_more  

2016 data

In [7]:
city_census.columns

Index(['Census_ID', 'city_id', 'median_age', 'male', 'female', 'white_alone',
       'black_africa', 'native', 'asian', 'islander', 'other',
       'median_family_income', 'household_income', 'median_household_income',
       'household_less_20', 'household_20_39', 'household_40_59',
       'household_60_99', 'household_100_149', 'household_150_200',
       'household_200_more', 'below_poverty', 'below_poverty_under_18',
       'below_poverty_18_64', 'below_poverty_65_over', 'above_poverty',
       'above_poverty_65_over', 'travel_time', 'travel_time_less_10',
       'travel_time_10_29', 'travel_time_30_59', 'travel_time_60_more',
       'means_transit', 'means_transit_drove_alone', 'means_transit_other',
       'means_transit_worked_home', 'total_pop_in_occupied_housing',
       'owner_occupied', 'renter_occupied', 'total_occupancy', 'occupied',
       'vacant', 'total_housing_units', 'total_geographical_mobility',
       'same_house_1_year', 'moved_within_same_county',
       'moved_

In [8]:
city_census_2016 = city_census[city_census['year']==2016]
city_census_2016

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
0,4026,1608,35.8,33576,38782,61904.0,4394.0,321.0,2502.0,99.0,915.0,78288.0,29102.0,56808.0,4564.0,5569.0,5244.0,6888.0,3839.0,1809.0,1189.0,8868.0,2680.0,5911.0,277.0,60917.0,9832.0,38103.0,6261.0,21407.0,9527.0,908.0,39388.0,33460.0,1154.0,1285.0,69543,44772,24771,30865,29102,1763,30865.0,71931.0,59910.0,7309.0,2672.0,1492.0,980.0,2616.0,12230.0,10073.0,10970.0,5610.0,72358.0,68312.0,52445.0,15374.0,493.0,2016
1,4027,1609,37.8,19178,19694,31181.0,185.0,238.0,5405.0,0.0,221.0,53878.0,16312.0,41226.0,2974.0,5028.0,3068.0,3772.0,793.0,254.0,423.0,8595.0,3318.0,4650.0,627.0,28858.0,5616.0,16982.0,4137.0,10951.0,1749.0,145.0,17514.0,14636.0,735.0,532.0,37885,23368,14517,17679,16312,1367,17679.0,38442.0,32713.0,3804.0,1506.0,419.0,1049.0,1449.0,8830.0,4676.0,4528.0,1922.0,38872.0,36297.0,26777.0,9300.0,220.0,2016
2,4028,1610,37.9,23856,24084,41235.0,1555.0,324.0,2391.0,0.0,430.0,102471.0,19604.0,79384.0,2059.0,2362.0,3236.0,4377.0,3559.0,1981.0,2030.0,2000.0,0.0,1244.0,756.0,45384.0,6217.0,24137.0,3326.0,16261.0,4073.0,477.0,25370.0,21049.0,762.0,1233.0,47227,34438,12789,20290,19604,686,20290.0,46960.0,40526.0,3261.0,1316.0,1491.0,87.0,623.0,4898.0,5210.0,11501.0,8611.0,47940.0,44959.0,35093.0,9629.0,237.0,2016
3,4029,1611,38.5,29889,30197,48892.0,3334.0,971.0,1395.0,0.0,3252.0,66069.0,28055.0,49243.0,5448.0,5515.0,5701.0,7779.0,2786.0,446.0,380.0,7711.0,1635.0,4954.0,1122.0,51743.0,6924.0,31057.0,3942.0,18947.0,6971.0,1197.0,31637.0,26747.0,864.0,580.0,59256,34132,25124,29761,28055,1706,29761.0,59617.0,53169.0,4905.0,845.0,297.0,1109.0,2228.0,15992.0,9769.0,7945.0,3063.0,60086.0,56511.0,48312.0,7344.0,855.0,2016
4,4030,1612,40.3,15215,16486,29207.0,472.0,0.0,341.0,0.0,854.0,71615.0,13259.0,59445.0,1611.0,2326.0,2722.0,3459.0,1835.0,995.0,311.0,1920.0,410.0,897.0,613.0,29486.0,5907.0,15304.0,3581.0,5912.0,5088.0,723.0,15898.0,13366.0,130.0,594.0,31385,22033,9352,14128,13259,869,14128.0,31448.0,27670.0,2150.0,1032.0,596.0,662.0,667.0,6730.0,4977.0,4718.0,1563.0,31701.0,30759.0,26656.0,3988.0,115.0,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,4495,2325,27.4,13633,12822,6863.0,1246.0,0.0,62.0,0.0,17383.0,68343.0,3892.0,88613.0,264.0,201.0,480.0,1688.0,452.0,386.0,421.0,1919.0,659.0,1148.0,112.0,24280.0,1325.0,11663.0,692.0,7459.0,2692.0,820.0,11663.0,6299.0,985.0,0.0,26418,16985,9433,4287,3892,395,NaN,NaN,NaN,NaN,NaN,NaN,3777.0,1636.0,4302.0,3193.0,835.0,219.0,26455.0,14892.0,13390.0,749.0,753.0,2016
470,4496,1872,44.7,14644,15963,NaN,NaN,NaN,NaN,NaN,NaN,72315.0,13223.0,59712.0,2253.0,2301.0,2088.0,3703.0,2125.0,463.0,290.0,2645.0,866.0,1527.0,252.0,27888.0,5134.0,14245.0,1819.0,9154.0,3095.0,177.0,14634.0,13134.0,265.0,389.0,30454,23065,7389,14348,13223,1125,14348.0,29806.0,26866.0,1475.0,1283.0,182.0,457.0,639.0,6990.0,4829.0,3519.0,2590.0,30607.

In [9]:
city_census_2016.isnull().sum()

Census_ID                                   0  
city_id                                     0  
median_age                                  0  
male                                        0  
female                                      0  
white_alone                                 130
black_africa                                130
native                                      130
asian                                       130
islander                                    130
other                                       130
median_family_income                        0  
household_income                            0  
median_household_income                     0  
household_less_20                           0  
household_20_39                             0  
household_40_59                             0  
household_60_99                             0  
household_100_149                           0  
household_150_200                           0  
household_200_more                      

In [10]:
city_census_2016.describe()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
count,474.000000,474.000000,474.000000,4.740000e+02,4.740000e+02,3.440000e+02,3.440000e+02,344.000000,3.440000e+02,344.000000,3.440000e+02,474.000000,4.740000e+02,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,4.740000e+02,474.000000,474.000000,474.000000,4.740000e+02,474.000000,4.740000e+02,474.00000,474.000000,4.740000e+02,4.740000e+02,4.470000e+02,447.000000,447.000000,447.000000,4.740000e+02,4.740000e+02,4.740000e+02,4.740000e+02,4.740000e+02,474.000000,4.060000e+02,4.000000e+02,4.000000e+02,400.000000,400.000000,400.000000,474.000000,474.000000,4.740000e+02,474.000000,4.740000e+02,474.000000,4.600000e+02,4.600000e+02,4.600000e+02,460.000000,460.000000,474.0
mean,4262.500000,1637.071730,36.601266,4.691707e+04,4.929154e+04,7.600326e+04,2.179207e+04,756.558140,8.947328e+03,145.305233,9.997622e+03,76190.318565,3.554945e+04,63256.320675,6558.888186,6888.529536,5662.947257,7569.814346,4676.666667,1950.364979,2242.238397,1.592501e+04,5308.432489,9118.877637,1497.702532,7.827554e+04,10578.421941,4.386884e+04,4507.14557,21607.170886,1.315539e+04,4.599135e+03,4.792784e+04,32275.780761,2734.480984,2064.024609,9.409585e+04,5.054849e+04,4.354736e+04,3.941329e+04,3.554945e+04,3863.843882,4.428626e+04,1.056312e+05,8.900901e+04,9893.302500,3199.855000,2554.787500,4645.042194,4809.755274,1.577984e+04,12421.253165,1.328124e+04,8005.542194,9.659659e+04,7.663138e+04,5.234283e+04,22369.519565,1919.030435,2016.0
std,136.976275,445.333325,5.273272,2.010311e+05,2.178261e+05,2.222915e+05,1.199099e+05,2552.074447,6.600282e+04,499.129332,7.121095e+04,30202.848181,1.536234e+05,26140.071086,31443.217290,27364.064373,21065.654863,30569.476325,20494.383559,9902.055668,13901.557594,7.876703e+04,24610.456722,44794.152883,9750.802579,3.338052e+05,44038.115858,1.889910e+05,9280.84413,58316.152154,8.058746e+04,4.772089e+04,2.033261e+05,72413.150566,24157.006872,9010.037678,4.103842e+05,1.526033e+05,2.608845e+05,1.708908e+05,1.536234e+05,17455.286232,1.842253e+05,4.489659e+05,4.000027e+05,30397.548158,9179.284724,7130.310977,28127.760535,26062.033398,6.968192e+04,42735.722181,6.244647e+04,44115.348380,4.248936e+05,2.740422e+05,2.063825e+05,58620.122443,14227.629864,0.0
min,4026.000000,629.000000,13.600000,8.350000e+03,7.940000e+03,2.425000e+03,5.900000e+01,0.000000,0.000000e+00,0.000000,0.000000e+00,18832.000000,3.514000e+03,12516.000000,66.000000,201.000000,69.000000,295.000000,34.000000,0.000000,0.000000,2.050000e+02,0.000000,56.000000,0.000000,1.095000e+04,51.000000,3.564000e+03,143.00000,1696.000000,1.090000e+02,0.000000e+00,3.811000e+03,1350.000000,0.000000,0.000000,1.366100e+04,1.130000e+02,2.180000e+02,3.918000e+03,3.514000e+03,0.000000,3.918000e+03,1.886800e+04,1.308500e+04,89.000000,0.000000,0.000000,0.000000,18.000000,6.270000e+02,637.000000,2.120000e+02,34.000000,1.960900e+04,1.255300e+04,2.555000e+03,506.000000,0.000000,2016.0
25%,4144.250000,1349.250000,33.300000,1.267000e+04,1.317225e+04,2.095425e+04,1.732750e+03,7.000000,

In [11]:
city_census_2016 = city_census_2016.apply(lambda x: x.fillna(x.mean()),axis=0)

In [12]:
city_census_2016.isnull().sum()

Census_ID                                   0
city_id                                     0
median_age                                  0
male                                        0
female                                      0
white_alone                                 0
black_africa                                0
native                                      0
asian                                       0
islander                                    0
other                                       0
median_family_income                        0
household_income                            0
median_household_income                     0
household_less_20                           0
household_20_39                             0
household_40_59                             0
household_60_99                             0
household_100_149                           0
household_150_200                           0
household_200_more                          0
below_poverty                     

In [13]:
city_census_2016.describe()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
count,474.000000,474.000000,474.000000,4.740000e+02,4.740000e+02,4.740000e+02,4.740000e+02,474.000000,4.740000e+02,474.000000,4.740000e+02,474.000000,4.740000e+02,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,4.740000e+02,474.000000,474.000000,474.000000,4.740000e+02,474.000000,4.740000e+02,474.00000,474.000000,4.740000e+02,4.740000e+02,4.740000e+02,474.000000,474.000000,474.000000,4.740000e+02,4.740000e+02,4.740000e+02,4.740000e+02,4.740000e+02,474.000000,4.740000e+02,4.740000e+02,4.740000e+02,474.000000,474.000000,474.00000,474.000000,474.000000,4.740000e+02,474.000000,4.740000e+02,474.000000,4.740000e+02,4.740000e+02,4.740000e+02,474.000000,474.000000,474.0
mean,4262.500000,1637.071730,36.601266,4.691707e+04,4.929154e+04,7.600326e+04,2.179207e+04,756.558140,8.947328e+03,145.305233,9.997622e+03,76190.318565,3.554945e+04,63256.320675,6558.888186,6888.529536,5662.947257,7569.814346,4676.666667,1950.364979,2242.238397,1.592501e+04,5308.432489,9118.877637,1497.702532,7.827554e+04,10578.421941,4.386884e+04,4507.14557,21607.170886,1.315539e+04,4.599135e+03,4.792784e+04,32275.780761,2734.480984,2064.024609,9.409585e+04,5.054849e+04,4.354736e+04,3.941329e+04,3.554945e+04,3863.843882,4.428626e+04,1.056312e+05,8.900901e+04,9893.302500,3199.855000,2554.78750,4645.042194,4809.755274,1.577984e+04,12421.253165,1.328124e+04,8005.542194,9.659659e+04,7.663138e+04,5.234283e+04,22369.519565,1919.030435,2016.0
std,136.976275,445.333325,5.273272,2.010311e+05,2.178261e+05,1.892951e+05,1.021108e+05,2173.250703,5.620552e+04,425.039784,6.064057e+04,30202.848181,1.536234e+05,26140.071086,31443.217290,27364.064373,21065.654863,30569.476325,20494.383559,9902.055668,13901.557594,7.876703e+04,24610.456722,44794.152883,9750.802579,3.338052e+05,44038.115858,1.889910e+05,9280.84413,58316.152154,8.058746e+04,4.772089e+04,1.974377e+05,70316.023441,23457.405847,8749.101726,4.103842e+05,1.526033e+05,2.608845e+05,1.708908e+05,1.536234e+05,17455.286232,1.704693e+05,4.123530e+05,3.673827e+05,27918.651183,8430.721023,6548.83953,28127.760535,26062.033398,6.968192e+04,42735.722181,6.244647e+04,44115.348380,4.185583e+05,2.699562e+05,2.033053e+05,57746.078066,14015.491449,0.0
min,4026.000000,629.000000,13.600000,8.350000e+03,7.940000e+03,2.425000e+03,5.900000e+01,0.000000,0.000000e+00,0.000000,0.000000e+00,18832.000000,3.514000e+03,12516.000000,66.000000,201.000000,69.000000,295.000000,34.000000,0.000000,0.000000,2.050000e+02,0.000000,56.000000,0.000000,1.095000e+04,51.000000,3.564000e+03,143.00000,1696.000000,1.090000e+02,0.000000e+00,3.811000e+03,1350.000000,0.000000,0.000000,1.366100e+04,1.130000e+02,2.180000e+02,3.918000e+03,3.514000e+03,0.000000,3.918000e+03,1.886800e+04,1.308500e+04,89.000000,0.000000,0.00000,0.000000,18.000000,6.270000e+02,637.000000,2.120000e+02,34.000000,1.960900e+04,1.255300e+04,2.555000e+03,506.000000,0.000000,2016.0
25%,4144.250000,1349.250000,33.300000,1.267000e+04,1.317225e+04,2.490475e+04,2.508750e+03,63.500000,1.0

2017

In [14]:
city_census_2017 = city_census[city_census['year']==2017]
city_census_2017.isnull().sum()

Census_ID                                   0  
city_id                                     0  
median_age                                  0  
male                                        0  
female                                      0  
white_alone                                 573
black_africa                                573
native                                      573
asian                                       573
islander                                    573
other                                       573
median_family_income                        0  
household_income                            0  
median_household_income                     0  
household_less_20                           0  
household_20_39                             0  
household_40_59                             0  
household_60_99                             0  
household_100_149                           0  
household_150_200                           0  
household_200_more                      

In [15]:
city_census_2017.describe()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
count,2302.000000,2302.000000,2302.000000,2.302000e+03,2.302000e+03,1.729000e+03,1.729000e+03,1729.000000,1.729000e+03,1729.000000,1.729000e+03,2302.000000,2.302000e+03,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2.296000e+03,2296.000000,2296.000000,2296.000000,2.296000e+03,2296.000000,2.301000e+03,2301.000000,2301.000000,2.301000e+03,2.301000e+03,2.178000e+03,2.178000e+03,2178.000000,2178.000000,2.302000e+03,2.302000e+03,2.302000e+03,2.302000e+03,2.302000e+03,2302.000000,1.938000e+03,2.041000e+03,2.041000e+03,2041.000000,2041.000000,2041.000000,2302.000000,2302.000000,2.302000e+03,2302.000000,2.302000e+03,2302.000000,2.288000e+03,2.288000e+03,2.288000e+03,2288.000000,2288.000000,2302.0
mean,5650.500000,1179.738488,37.362294,3.759433e+04,3.928165e+04,5.839039e+04,1.509686e+04,607.864083,7.777854e+03,231.766339,7.296113e+03,81379.513467,2.814158e+04,67914.412685,4640.986968,5108.541268,4380.108601,6126.163336,4008.790182,1810.537359,2066.450912,1.128576e+04,3557.937282,6609.129355,1118.689460,6.388546e+04,9257.790941,3.500959e+04,3707.261625,17767.982182,1.026486e+04,3.269488e+03,3.838181e+04,2.780214e+04,2122.381084,1978.068871,7.495297e+04,4.291953e+04,3.203345e+04,3.125173e+04,2.814158e+04,3110.155951,3.510746e+04,8.162493e+04,6.833359e+04,7781.150906,2611.568349,2158.522293,3126.807993,3577.943093,1.238977e+04,10332.210686,1.098505e+04,6938.428758,7.684481e+04,6.231469e+04,4.101265e+04,19798.744755,1503.289773,2017.0
std,664.674482,686.186563,6.079030,1.152593e+05,1.229995e+05,1.347973e+05,6.658777e+04,1967.265010,3.681555e+04,872.726491,4.198124e+04,33587.115120,8.738719e+04,28956.528450,17300.688319,15281.668635,12517.603245,17346.936612,12061.522411,6056.009596,8596.744055,4.261162e+04,13186.097786,24515.683059,5280.292785,1.926287e+05,26181.810475,1.089966e+05,6269.408883,40837.660702,4.411075e+04,2.428859e+04,1.175690e+05,5.839744e+04,12992.327229,5941.857543,2.332661e+05,9.778925e+04,1.392253e+05,9.719271e+04,8.738719e+04,10333.307907,1.054620e+05,2.491634e+05,2.189517e+05,20347.572517,5412.623051,5053.038367,15853.169745,14428.400003,3.934502e+04,26523.567628,3.659375e+04,25122.682781,2.388458e+05,1.646424e+05,1.205900e+05,43098.712172,7356.306721,0.0
min,4500.000000,1.000000,13.200000,7.957000e+03,6.500000e+03,1.124000e+03,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,18168.000000,3.309000e+03,12400.000000,0.000000,85.000000,161.000000,201.000000,0.000000,0.000000,0.000000,6.400000e+01,0.000000,0.000000,0.000000,5.627000e+03,169.000000,1.918000e+03,40.000000,643.000000,1.040000e+02,0.000000e+00,2.360000e+03,1.476000e+03,0.000000,0.000000,7.967000e+03,1.950000e+02,5.560000e+02,3.982000e+03,3.309000e+03,0.000000,4.107000e+03,1.860500e+04,6.097000e+03,69.000000,0.000000,0.000000,0.000000,0.000000,2.650000e+02,388.000000,1.820000e+02,0.000000,1.905800e+04,5.654000e+03,1.273000e+03,216.000000,0.000000,2017.0
25%,5075.250000,586.250000,33.900000,1.280275e+04,1.332175e+04,2.04980

In [16]:
city_census_2017 = city_census_2017.apply(lambda x: x.fillna(x.mean()),axis=0)
city_census_2017.isnull().sum()

Census_ID                                   0
city_id                                     0
median_age                                  0
male                                        0
female                                      0
white_alone                                 0
black_africa                                0
native                                      0
asian                                       0
islander                                    0
other                                       0
median_family_income                        0
household_income                            0
median_household_income                     0
household_less_20                           0
household_20_39                             0
household_40_59                             0
household_60_99                             0
household_100_149                           0
household_150_200                           0
household_200_more                          0
below_poverty                     

In [17]:
city_census_2017.describe()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
count,2302.000000,2302.000000,2302.000000,2.302000e+03,2.302000e+03,2.302000e+03,2.302000e+03,2302.000000,2.302000e+03,2302.000000,2.302000e+03,2302.000000,2.302000e+03,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2.302000e+03,2302.000000,2302.000000,2302.000000,2.302000e+03,2302.000000,2.302000e+03,2302.000000,2302.000000,2.302000e+03,2.302000e+03,2.302000e+03,2.302000e+03,2302.000000,2302.000000,2.302000e+03,2.302000e+03,2.302000e+03,2.302000e+03,2.302000e+03,2302.000000,2.302000e+03,2.302000e+03,2.302000e+03,2302.000000,2302.000000,2302.000000,2302.000000,2302.000000,2.302000e+03,2302.000000,2.302000e+03,2302.000000,2.302000e+03,2.302000e+03,2.302000e+03,2302.000000,2302.000000,2302.0
mean,5650.500000,1179.738488,37.362294,3.759433e+04,3.928165e+04,5.839039e+04,1.509686e+04,607.864083,7.777854e+03,231.766339,7.296113e+03,81379.513467,2.814158e+04,67914.412685,4640.986968,5108.541268,4380.108601,6126.163336,4008.790182,1810.537359,2066.450912,1.128576e+04,3557.937282,6609.129355,1118.689460,6.388546e+04,9257.790941,3.500959e+04,3707.261625,17767.982182,1.026486e+04,3.269488e+03,3.838181e+04,2.780214e+04,2122.381084,1978.068871,7.495297e+04,4.291953e+04,3.203345e+04,3.125173e+04,2.814158e+04,3110.155951,3.510746e+04,8.162493e+04,6.833359e+04,7781.150906,2611.568349,2158.522293,3126.807993,3577.943093,1.238977e+04,10332.210686,1.098505e+04,6938.428758,7.684481e+04,6.231469e+04,4.101265e+04,19798.744755,1503.289773,2017.0
std,664.674482,686.186563,6.079030,1.152593e+05,1.229995e+05,1.168140e+05,5.770428e+04,1704.811739,3.190398e+04,756.295853,3.638051e+04,33587.115120,8.738719e+04,28956.528450,17300.688319,15281.668635,12517.603245,17346.936612,12061.522411,6056.009596,8596.744055,4.255603e+04,13168.894781,24483.699115,5273.403946,1.923773e+05,26147.652847,1.089730e+05,6268.046412,40828.785843,4.410117e+04,2.428331e+04,1.143572e+05,5.680215e+04,12637.403593,5779.538226,2.332661e+05,9.778925e+04,1.392253e+05,9.719271e+04,8.738719e+04,10333.307907,9.676152e+04,2.346070e+05,2.061603e+05,19158.847445,5096.412322,4757.834927,15853.169745,14428.400003,3.934502e+04,26523.567628,3.659375e+04,25122.682781,2.381181e+05,1.641407e+05,1.202226e+05,42967.399142,7333.893547,0.0
min,4500.000000,1.000000,13.200000,7.957000e+03,6.500000e+03,1.124000e+03,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,18168.000000,3.309000e+03,12400.000000,0.000000,85.000000,161.000000,201.000000,0.000000,0.000000,0.000000,6.400000e+01,0.000000,0.000000,0.000000,5.627000e+03,169.000000,1.918000e+03,40.000000,643.000000,1.040000e+02,0.000000e+00,2.360000e+03,1.476000e+03,0.000000,0.000000,7.967000e+03,1.950000e+02,5.560000e+02,3.982000e+03,3.309000e+03,0.000000,4.107000e+03,1.860500e+04,6.097000e+03,69.000000,0.000000,0.000000,0.000000,0.000000,2.650000e+02,388.000000,1.820000e+02,0.000000,1.905800e+04,5.654000e+03,1.273000e+03,216.000000,0.000000,2017.0
25%,5075.250000,586.250000,33.900000,1.280275e+04,1.332175e+04,2.38480

2018

In [18]:
city_census_2018 = city_census[city_census['year']==2018]
city_census_2018.isnull().sum()

Census_ID                                   0  
city_id                                     0  
median_age                                  0  
male                                        0  
female                                      0  
white_alone                                 585
black_africa                                585
native                                      585
asian                                       585
islander                                    585
other                                       585
median_family_income                        1  
household_income                            1  
median_household_income                     0  
household_less_20                           1  
household_20_39                             1  
household_40_59                             1  
household_60_99                             1  
household_100_149                           1  
household_150_200                           1  
household_200_more                      

In [19]:
city_census_2018.describe()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
count,2323.00000,2323.000000,2323.000000,2.323000e+03,2.323000e+03,1.738000e+03,1.738000e+03,1738.000000,1.738000e+03,1738.000000,1.738000e+03,2322.000000,2.322000e+03,2323.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2.317000e+03,2317.000000,2317.000000,2317.000000,2.317000e+03,2317.000000,2.322000e+03,2322.000000,2322.000000,2.322000e+03,2.322000e+03,2.202000e+03,2.202000e+03,2202.000000,2202.000000,2.323000e+03,2.323000e+03,2.323000e+03,2.323000e+03,2.323000e+03,2323.000000,1.951000e+03,2.019000e+03,2.019000e+03,2019.000000,2019.000000,2019.000000,2322.000000,2322.000000,2.322000e+03,2322.000000,2.322000e+03,2322.000000,2.308000e+03,2.308000e+03,2.308000e+03,2308.000000,2308.000000,2323.0
mean,7963.00000,1194.690056,37.474559,3.758594e+04,3.922677e+04,5.850587e+04,1.512458e+04,643.727848,7.815506e+03,231.679517,7.089475e+03,84693.705426,2.836324e+04,70418.186397,4471.314815,5000.228682,4342.004737,6156.326012,4166.517227,1910.394488,2316.457795,1.095981e+04,3402.999568,6385.314631,1171.495037,6.414909e+04,9490.217523,3.528675e+04,3689.586563,17821.122739,1.044057e+04,3.335473e+03,3.864963e+04,2.800341e+04,2138.335604,2015.767484,7.488986e+04,4.309765e+04,3.179221e+04,3.139968e+04,2.835395e+04,3045.724064,3.533101e+04,8.273587e+04,6.941657e+04,7765.394255,2656.840020,2221.308569,3060.152024,3483.645134,1.228775e+04,10249.837640,1.122005e+04,7162.245478,7.682506e+04,6.237406e+04,4.087021e+04,19959.860052,1543.983969,2018.0
std,670.73666,695.202754,6.143975,1.134403e+05,1.207254e+05,1.336861e+05,6.527595e+04,2210.754319,3.574066e+04,929.823280,3.972546e+04,34769.388750,8.796189e+04,30112.753755,16468.851788,15100.806446,12352.390623,17509.733415,12469.328687,6310.999897,9549.411734,4.052715e+04,12460.363738,22973.141088,5527.954753,1.905787e+05,26339.857779,1.080447e+05,6279.370804,40822.749603,4.409166e+04,2.343501e+04,1.166125e+05,5.890425e+04,12673.688690,6056.602067,2.290006e+05,9.699457e+04,1.356433e+05,9.762276e+04,8.794409e+04,10162.446675,1.060500e+05,2.471706e+05,2.169859e+05,20295.763408,5550.275907,5136.486587,15635.213182,13344.922634,3.812418e+04,26115.291054,3.740550e+04,25609.699045,2.347809e+05,1.625355e+05,1.178119e+05,43780.234900,7313.268202,0.0
min,6802.00000,1.000000,13.200000,8.418000e+03,6.004000e+03,4.060000e+02,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,16009.000000,2.230000e+03,11762.000000,0.000000,0.000000,133.000000,195.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,3.770000e+03,36.000000,2.222000e+03,0.000000,644.000000,8.300000e+01,0.000000e+00,2.652000e+03,1.524000e+03,0.000000,0.000000,6.758000e+03,1.380000e+02,1.820000e+02,3.478000e+03,2.230000e+03,0.000000,3.478000e+03,1.861700e+04,5.559000e+03,127.000000,0.000000,0.000000,0.000000,0.000000,2.390000e+02,467.000000,4.400000e+01,0.000000,1.902300e+04,6.414000e+03,3.010000e+02,185.000000,0.000000,2018.0
25%,7382.50000,594.500000,33.900000,1.275100e+04,1.345800e+04,2.091450e+04,1

In [20]:
city_census_2018 = city_census_2018.apply(lambda x: x.fillna(x.mean()),axis=0)
city_census_2018.isnull().sum()

Census_ID                                   0
city_id                                     0
median_age                                  0
male                                        0
female                                      0
white_alone                                 0
black_africa                                0
native                                      0
asian                                       0
islander                                    0
other                                       0
median_family_income                        0
household_income                            0
median_household_income                     0
household_less_20                           0
household_20_39                             0
household_40_59                             0
household_60_99                             0
household_100_149                           0
household_150_200                           0
household_200_more                          0
below_poverty                     

In [21]:
city_census_2018.describe()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
count,2323.00000,2323.000000,2323.000000,2.323000e+03,2.323000e+03,2.323000e+03,2.323000e+03,2323.000000,2.323000e+03,2323.000000,2.323000e+03,2323.000000,2.323000e+03,2323.000000,2323.000000,2323.000000,2323.000000,2323.000000,2323.000000,2323.000000,2323.000000,2.323000e+03,2323.000000,2323.000000,2323.000000,2.323000e+03,2323.000000,2.323000e+03,2323.000000,2323.000000,2.323000e+03,2.323000e+03,2.323000e+03,2.323000e+03,2323.000000,2323.000000,2.323000e+03,2.323000e+03,2.323000e+03,2.323000e+03,2.323000e+03,2323.000000,2.323000e+03,2.323000e+03,2.323000e+03,2323.000000,2323.00000,2323.000000,2323.000000,2323.000000,2.323000e+03,2323.000000,2.323000e+03,2323.000000,2.323000e+03,2.323000e+03,2.323000e+03,2323.000000,2323.000000,2323.0
mean,7963.00000,1194.690056,37.474559,3.758594e+04,3.922677e+04,5.850587e+04,1.512458e+04,643.727848,7.815506e+03,231.679517,7.089475e+03,84693.705426,2.836324e+04,70418.186397,4471.314815,5000.228682,4342.004737,6156.326012,4166.517227,1910.394488,2316.457795,1.095981e+04,3402.999568,6385.314631,1171.495037,6.414909e+04,9490.217523,3.528675e+04,3689.586563,17821.122739,1.044057e+04,3.335473e+03,3.864963e+04,2.800341e+04,2138.335604,2015.767484,7.488986e+04,4.309765e+04,3.179221e+04,3.139968e+04,2.835395e+04,3045.724064,3.533101e+04,8.273587e+04,6.941657e+04,7765.394255,2656.84002,2221.308569,3060.152024,3483.645134,1.228775e+04,10249.837640,1.122005e+04,7162.245478,7.682506e+04,6.237406e+04,4.087021e+04,19959.860052,1543.983969,2018.0
std,670.73666,695.202754,6.143975,1.134403e+05,1.207254e+05,1.156259e+05,5.645755e+04,1912.094198,3.091230e+04,804.209533,3.435878e+04,34761.900996,8.794295e+04,30112.753755,16465.305142,15097.554415,12349.730477,17505.962610,12466.643357,6309.640793,9547.355222,4.047475e+04,12444.254663,22943.440829,5520.808073,1.903324e+05,26305.804943,1.080214e+05,6278.018511,40813.958228,4.408217e+04,2.342996e+04,1.135335e+05,5.734896e+04,12339.056382,5896.685347,2.290006e+05,9.699457e+04,1.356433e+05,9.762276e+04,8.794409e+04,10162.446675,9.718450e+04,2.304233e+05,2.022837e+05,18920.598276,5174.20994,4788.457438,15631.846064,13342.048741,3.811597e+04,26109.667001,3.739744e+04,25604.183873,2.340214e+05,1.620096e+05,1.174308e+05,43638.596758,7289.608262,0.0
min,6802.00000,1.000000,13.200000,8.418000e+03,6.004000e+03,4.060000e+02,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,16009.000000,2.230000e+03,11762.000000,0.000000,0.000000,133.000000,195.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,3.770000e+03,36.000000,2.222000e+03,0.000000,644.000000,8.300000e+01,0.000000e+00,2.652000e+03,1.524000e+03,0.000000,0.000000,6.758000e+03,1.380000e+02,1.820000e+02,3.478000e+03,2.230000e+03,0.000000,3.478000e+03,1.861700e+04,5.559000e+03,127.000000,0.00000,0.000000,0.000000,0.000000,2.390000e+02,467.000000,4.400000e+01,0.000000,1.902300e+04,6.414000e+03,3.010000e+02,185.000000,0.000000,2018.0
25%,7382.50000,594.500000,33.900000,1.275100e+04,1.345800e+04,2.414850e+04,2.059

2019

In [22]:
city_census_2019 = city_census[city_census['year']==2019]
city_census_2019.isnull().sum()

Census_ID                                   0  
city_id                                     0  
median_age                                  0  
male                                        0  
female                                      0  
white_alone                                 627
black_africa                                627
native                                      627
asian                                       627
islander                                    627
other                                       627
median_family_income                        1  
household_income                            3  
median_household_income                     1  
household_less_20                           3  
household_20_39                             3  
household_40_59                             3  
household_60_99                             3  
household_100_149                           3  
household_150_200                           3  
household_200_more                      

In [23]:
city_census_2019.describe()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
count,2333.000000,2333.000000,2333.000000,2.333000e+03,2.333000e+03,1.706000e+03,1.706000e+03,1706.000000,1.706000e+03,1706.000000,1.706000e+03,2332.000000,2.330000e+03,2332.000000,2330.000000,2330.000000,2330.000000,2330.000000,2330.000000,2330.000000,2330.000000,2.328000e+03,2328.000000,2328.000000,2328.000000,2.328000e+03,2.328000e+03,2.333000e+03,2333.000000,2333.000000,2.333000e+03,2.333000e+03,2.206000e+03,2.206000e+03,2206.000000,2206.000000,2.333000e+03,2.333000e+03,2.333000e+03,2.333000e+03,2.333000e+03,2333.000000,1.917000e+03,1.978000e+03,1.978000e+03,1978.000000,1978.000000,1978.000000,2332.000000,2332.000000,2.332000e+03,2332.000000,2.332000e+03,2332.000000,2.317000e+03,2.317000e+03,2.317000e+03,2317.000000,2317.000000,2333.0
mean,10291.000000,1205.258465,37.691127,3.757915e+04,3.923830e+04,5.937856e+04,1.527465e+04,678.970692,7.974611e+03,239.834115,7.178826e+03,89458.866638,2.859339e+04,74540.648799,4158.376395,4745.041631,4301.440343,6303.017597,4402.970815,2085.196567,2597.349785,1.025258e+04,3106.661512,5935.731529,1210.183849,6.478735e+04,9.762131e+03,3.558467e+04,3711.890699,17836.321903,1.057395e+04,3.462505e+03,3.923827e+04,2.828034e+04,2216.057117,2230.047597,7.488694e+04,4.338544e+04,3.150149e+04,3.160066e+04,2.856554e+04,3035.119160,3.610142e+04,8.409822e+04,7.090027e+04,7652.835187,2674.777048,2204.963094,2980.392796,3405.586192,1.236665e+04,10148.617067,1.145834e+04,7330.172384,7.685808e+04,6.248956e+04,4.097685e+04,19966.911524,1545.791109,2019.0
std,673.623411,703.948294,6.080667,1.127310e+05,1.200915e+05,1.348671e+05,6.609598e+04,2201.171667,3.627594e+04,1023.940666,3.917549e+04,36233.077601,8.870613e+04,31557.353629,15544.287245,14134.412426,12196.736047,17865.148325,13063.385887,6941.989164,10896.565637,3.778506e+04,11470.062577,21053.271534,5658.480482,1.916588e+05,2.703420e+04,1.086974e+05,6377.082160,40757.007614,4.437115e+04,2.337768e+04,1.178694e+05,5.933545e+04,13375.251147,6550.768510,2.274400e+05,9.593271e+04,1.355051e+05,9.824817e+04,8.865245e+04,10022.463149,1.078373e+05,2.487148e+05,2.178735e+05,20711.227256,5446.809598,5151.019951,14989.340870,13023.772052,3.832454e+04,25652.759179,3.776684e+04,26108.794656,2.334852e+05,1.629228e+05,1.181714e+05,43900.356852,7340.815397,0.0
min,9125.000000,1.000000,14.100000,7.861000e+03,4.944000e+03,1.135000e+03,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,20321.000000,2.076000e+03,13649.000000,0.000000,0.000000,87.000000,320.000000,0.000000,0.000000,0.000000,5.400000e+01,0.000000,0.000000,0.000000,4.987000e+03,0.000000e+00,2.741000e+03,0.000000,535.000000,1.860000e+02,0.000000e+00,3.470000e+03,1.930000e+03,0.000000,0.000000,7.487000e+03,7.700000e+01,1.370000e+02,3.536000e+03,2.076000e+03,0.000000,4.252000e+03,1.840500e+04,6.841000e+03,111.000000,0.000000,0.000000,0.000000,0.000000,2.750000e+02,625.000000,2.260000e+02,0.000000,1.900900e+04,5.793000e+03,2.710000e+02,308.000000,0.000000,2019.0
25%,9708.000000,600.000000,34.300000,1.278500e+04,1.338400e+04,2.

In [24]:
city_census_2019 = city_census_2019.apply(lambda x: x.fillna(x.mean()),axis=0)
city_census_2019.isnull().sum()

Census_ID                                   0
city_id                                     0
median_age                                  0
male                                        0
female                                      0
white_alone                                 0
black_africa                                0
native                                      0
asian                                       0
islander                                    0
other                                       0
median_family_income                        0
household_income                            0
median_household_income                     0
household_less_20                           0
household_20_39                             0
household_40_59                             0
household_60_99                             0
household_100_149                           0
household_150_200                           0
household_200_more                          0
below_poverty                     

In [25]:
city_census_2019.describe()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
count,2333.000000,2333.000000,2333.000000,2.333000e+03,2.333000e+03,2.333000e+03,2.333000e+03,2333.000000,2.333000e+03,2333.000000,2.333000e+03,2333.000000,2.333000e+03,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2.333000e+03,2333.000000,2333.000000,2333.000000,2.333000e+03,2.333000e+03,2.333000e+03,2333.000000,2333.000000,2.333000e+03,2.333000e+03,2.333000e+03,2.333000e+03,2333.000000,2333.000000,2.333000e+03,2.333000e+03,2.333000e+03,2.333000e+03,2.333000e+03,2333.000000,2.333000e+03,2.333000e+03,2.333000e+03,2333.000000,2333.000000,2333.000000,2333.000000,2333.000000,2.333000e+03,2333.000000,2.333000e+03,2333.000000,2.333000e+03,2.333000e+03,2.333000e+03,2333.000000,2333.000000,2333.0
mean,10291.000000,1205.258465,37.691127,3.757915e+04,3.923830e+04,5.937856e+04,1.527465e+04,678.970692,7.974611e+03,239.834115,7.178826e+03,89458.866638,2.859339e+04,74540.648799,4158.376395,4745.041631,4301.440343,6303.017597,4402.970815,2085.196567,2597.349785,1.025258e+04,3106.661512,5935.731529,1210.183849,6.478735e+04,9.762131e+03,3.558467e+04,3711.890699,17836.321903,1.057395e+04,3.462505e+03,3.923827e+04,2.828034e+04,2216.057117,2230.047597,7.488694e+04,4.338544e+04,3.150149e+04,3.160066e+04,2.856554e+04,3035.119160,3.610142e+04,8.409822e+04,7.090027e+04,7652.835187,2674.777048,2204.963094,2980.392796,3405.586192,1.236665e+04,10148.617067,1.145834e+04,7330.172384,7.685808e+04,6.248956e+04,4.097685e+04,19966.911524,1545.791109,2019.0
std,673.623411,703.948294,6.080667,1.127310e+05,1.200915e+05,1.153198e+05,5.651620e+04,1882.139602,3.101820e+04,875.533384,3.349749e+04,36225.308098,8.864905e+04,31550.586747,15534.285557,14125.317897,12188.888281,17853.653322,13054.980490,6937.522468,10889.554442,3.774453e+04,11457.759600,21030.689449,5652.411103,1.914532e+05,2.700520e+04,1.086974e+05,6377.082160,40757.007614,4.437115e+04,2.337768e+04,1.146149e+05,5.769714e+04,13005.946685,6369.895043,2.274400e+05,9.593271e+04,1.355051e+05,9.824817e+04,8.865245e+04,10022.463149,9.774678e+04,2.290028e+05,2.006058e+05,19069.745436,5015.119152,4742.772506,14986.126687,13020.979349,3.831633e+04,25647.258426,3.775874e+04,26103.196115,2.326828e+05,1.623629e+05,1.177654e+05,43749.496073,7315.589153,0.0
min,9125.000000,1.000000,14.100000,7.861000e+03,4.944000e+03,1.135000e+03,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,20321.000000,2.076000e+03,13649.000000,0.000000,0.000000,87.000000,320.000000,0.000000,0.000000,0.000000,5.400000e+01,0.000000,0.000000,0.000000,4.987000e+03,0.000000e+00,2.741000e+03,0.000000,535.000000,1.860000e+02,0.000000e+00,3.470000e+03,1.930000e+03,0.000000,0.000000,7.487000e+03,7.700000e+01,1.370000e+02,3.536000e+03,2.076000e+03,0.000000,4.252000e+03,1.840500e+04,6.841000e+03,111.000000,0.000000,0.000000,0.000000,0.000000,2.750000e+02,625.000000,2.260000e+02,0.000000,1.900900e+04,5.793000e+03,2.710000e+02,308.000000,0.000000,2019.0
25%,9708.000000,600.000000,34.300000,1.278500e+04,1.338400e+04,2.4

merging all the 4 separate data sets

In [26]:
frame_1 = [city_census_2016,city_census_2017,city_census_2018,city_census_2019]
city_census_cleaned = pd.concat(frame_1)
city_census_cleaned.head()

,Census_ID,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us,year
0,4026,1608,35.8,33576,38782,61904.0,4394.0,321.0,2502.0,99.0,915.0,78288.0,29102.0,56808.0,4564.0,5569.0,5244.0,6888.0,3839.0,1809.0,1189.0,8868.0,2680.0,5911.0,277.0,60917.0,9832.0,38103.0,6261.0,21407.0,9527.0,908.0,39388.0,33460.0,1154.0,1285.0,69543,44772,24771,30865,29102,1763,30865.0,71931.0,59910.0,7309.0,2672.0,1492.0,980.0,2616.0,12230.0,10073.0,10970.0,5610.0,72358.0,68312.0,52445.0,15374.0,493.0,2016
1,4027,1609,37.8,19178,19694,31181.0,185.0,238.0,5405.0,0.0,221.0,53878.0,16312.0,41226.0,2974.0,5028.0,3068.0,3772.0,793.0,254.0,423.0,8595.0,3318.0,4650.0,627.0,28858.0,5616.0,16982.0,4137.0,10951.0,1749.0,145.0,17514.0,14636.0,735.0,532.0,37885,23368,14517,17679,16312,1367,17679.0,38442.0,32713.0,3804.0,1506.0,419.0,1049.0,1449.0,8830.0,4676.0,4528.0,1922.0,38872.0,36297.0,26777.0,9300.0,220.0,2016
2,4028,1610,37.9,23856,24084,41235.0,1555.0,324.0,2391.0,0.0,430.0,102471.0,19604.0,79384.0,2059.0,2362.0,3236.0,4377.0,3559.0,1981.0,2030.0,2000.0,0.0,1244.0,756.0,45384.0,6217.0,24137.0,3326.0,16261.0,4073.0,477.0,25370.0,21049.0,762.0,1233.0,47227,34438,12789,20290,19604,686,20290.0,46960.0,40526.0,3261.0,1316.0,1491.0,87.0,623.0,4898.0,5210.0,11501.0,8611.0,47940.0,44959.0,35093.0,9629.0,237.0,2016
3,4029,1611,38.5,29889,30197,48892.0,3334.0,971.0,1395.0,0.0,3252.0,66069.0,28055.0,49243.0,5448.0,5515.0,5701.0,7779.0,2786.0,446.0,380.0,7711.0,1635.0,4954.0,1122.0,51743.0,6924.0,31057.0,3942.0,18947.0,6971.0,1197.0,31637.0,26747.0,864.0,580.0,59256,34132,25124,29761,28055,1706,29761.0,59617.0,53169.0,4905.0,845.0,297.0,1109.0,2228.0,15992.0,9769.0,7945.0,3063.0,60086.0,56511.0,48312.0,7344.0,855.0,2016
4,4030,1612,40.3,15215,16486,29207.0,472.0,0.0,341.0,0.0,854.0,71615.0,13259.0,59445.0,1611.0,2326.0,2722.0,3459.0,1835.0,995.0,311.0,1920.0,410.0,897.0,613.0,29486.0,5907.0,15304.0,3581.0,5912.0,5088.0,723.0,15898.0,13366.0,130.0,594.0,31385,22033,9352,14128,13259,869,14128.0,31448.0,27670.0,2150.0,1032.0,596.0,662.0,667.0,6730.0,4977.0,4718.0,1563.0,31701.0,30759.0,26656.0,3988.0,115.0,2016


In [27]:
city_census_cleaned.isnull().sum()

Census_ID                                   0
city_id                                     0
median_age                                  0
male                                        0
female                                      0
white_alone                                 0
black_africa                                0
native                                      0
asian                                       0
islander                                    0
other                                       0
median_family_income                        0
household_income                            0
median_household_income                     0
household_less_20                           0
household_20_39                             0
household_40_59                             0
household_60_99                             0
household_100_149                           0
household_150_200                           0
household_200_more                          0
below_poverty                     

In [28]:
city_census_cleaned = city_census_cleaned.drop('year',axis=1)

In [29]:
city_census_cleaned = city_census_cleaned.drop('Census_ID',axis=1)
city_census_cleaned.head()

,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us
0,1608,35.8,33576,38782,61904.0,4394.0,321.0,2502.0,99.0,915.0,78288.0,29102.0,56808.0,4564.0,5569.0,5244.0,6888.0,3839.0,1809.0,1189.0,8868.0,2680.0,5911.0,277.0,60917.0,9832.0,38103.0,6261.0,21407.0,9527.0,908.0,39388.0,33460.0,1154.0,1285.0,69543,44772,24771,30865,29102,1763,30865.0,71931.0,59910.0,7309.0,2672.0,1492.0,980.0,2616.0,12230.0,10073.0,10970.0,5610.0,72358.0,68312.0,52445.0,15374.0,493.0
1,1609,37.8,19178,19694,31181.0,185.0,238.0,5405.0,0.0,221.0,53878.0,16312.0,41226.0,2974.0,5028.0,3068.0,3772.0,793.0,254.0,423.0,8595.0,3318.0,4650.0,627.0,28858.0,5616.0,16982.0,4137.0,10951.0,1749.0,145.0,17514.0,14636.0,735.0,532.0,37885,23368,14517,17679,16312,1367,17679.0,38442.0,32713.0,3804.0,1506.0,419.0,1049.0,1449.0,8830.0,4676.0,4528.0,1922.0,38872.0,36297.0,26777.0,9300.0,220.0
2,1610,37.9,23856,24084,41235.0,1555.0,324.0,2391.0,0.0,430.0,102471.0,19604.0,79384.0,2059.0,2362.0,3236.0,4377.0,3559.0,1981.0,2030.0,2000.0,0.0,1244.0,756.0,45384.0,6217.0,24137.0,3326.0,16261.0,4073.0,477.0,25370.0,21049.0,762.0,1233.0,47227,34438,12789,20290,19604,686,20290.0,46960.0,40526.0,3261.0,1316.0,1491.0,87.0,623.0,4898.0,5210.0,11501.0,8611.0,47940.0,44959.0,35093.0,9629.0,237.0
3,1611,38.5,29889,30197,48892.0,3334.0,971.0,1395.0,0.0,3252.0,66069.0,28055.0,49243.0,5448.0,5515.0,5701.0,7779.0,2786.0,446.0,380.0,7711.0,1635.0,4954.0,1122.0,51743.0,6924.0,31057.0,3942.0,18947.0,6971.0,1197.0,31637.0,26747.0,864.0,580.0,59256,34132,25124,29761,28055,1706,29761.0,59617.0,53169.0,4905.0,845.0,297.0,1109.0,2228.0,15992.0,9769.0,7945.0,3063.0,60086.0,56511.0,48312.0,7344.0,855.0
4,1612,40.3,15215,16486,29207.0,472.0,0.0,341.0,0.0,854.0,71615.0,13259.0,59445.0,1611.0,2326.0,2722.0,3459.0,1835.0,995.0,311.0,1920.0,410.0,897.0,613.0,29486.0,5907.0,15304.0,3581.0,5912.0,5088.0,723.0,15898.0,13366.0,130.0,594.0,31385,22033,9352,14128,13259,869,14128.0,31448.0,27670.0,2150.0,1032.0,596.0,662.0,667.0,6730.0,4977.0,4718.0,1563.0,31701.0,30759.0,26656.0,3988.0,115.0


In [30]:
city_census_cleaned = city_census_cleaned.groupby('city_id').agg(['mean']).reset_index()
city_census_cleaned

,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
0,1,39.466667,16250.000000,17093.666667,35653.185619,8921.216295,253.323564,2716.536928,79.944705,4044.608636,84185.000000,11753.000000,71370.666667,979.333333,2059.666667,1755.333333,2977.666667,2853.666667,615.666667,511.666667,2403.666667,1007.000000,1109.333333,287.333333,30598.000000,3442.000000,15968.333333,913.666667,6281.000000,6560.666667,2213.000000,16833.666667,14500.000000,194.333333,865.333333,33060.000000,28059.666667,5000.333333,12355.000000,11753.000000,602.0,28135.627605,32926.000000,28319.333333,2359.333333,994.333333,1253.000000,435.666667,1414.666667,5021.000000,5556.000000,4824.333333,3030.000000,33343.666667,31417.000000,21250.666667,9668.000000,498.333333
1,2,31.833333,10686.666667,12834.333333,33403.128398,5423.620397,272.621361,2681.618084,77.255446,3066.037594,51624.666667,7725.333333,45917.666667,1768.000000,1642.666667,1496.333333,1500.666667,868.000000,204.666667,245.000000,5636.666667,2598.666667,2635.666667,402.333333,17547.333333,2289.000000,9479.666667,2790.000000,4838.000000,1048.333333,803.333333,19285.757328,14725.445905,781.019039,920.349199,23166.666667,12455.333333,10711.333333,8281.333333,7725.333333,556.0,8281.333333,43314.740647,37193.755308,3722.278396,1113.259016,1035.321031,1625.000000,1912.666667,3917.000000,2727.333333,1817.666667,679.000000,23521.000000,19905.333333,14678.666667,4918.666667,308.000000
2,3,42.500000,9926.666667,12434.333333,42305.141124,14362.408279,454.566180,5391.705705,157.171211,4780.433531,44740.666667,9854.333333,35290.666667,3243.666667,2175.666667,1660.666667,1200.333333,1025.666667,303.000000,245.333333,5453.333333,2018.666667,2967.000000,467.666667,15935.000000,3895.000000,8112.333333,1242.000000,4738.333333,1425.666667,706.333333,18893.090662,13399.779238,922.685706,1175.682532,21347.333333,11363.000000,9984.333333,12473.333333,9854.333333,2619.0,12473.333333,42824.073980,35706.755308,4435.611729,1408.592349,1019.654365,677.000000,1827.333333,5577.000000,3567.000000,1707.000000,1494.333333,22361.000000,21909.000000,16031.333333,5759.666667,118.000000
3,4,41.533333,12806.666667,13599.666667,19557.000000,4636.666667,218.666667,504.333333,76.333333,796.666667,75260.000000,9668.333333,51614.666667,2162.666667,1965.000000,1305.333333,1819.333333,1309.000000,551.000000,556.000000,4112.666667,1315.666667,2168.666667,628.333333,21711.000000,4170.666667,10436.333333,1954.333333,4062.000000,4109.000000,311.000000,38756.570592,28028.630960,2158.924602,2074.627984,25791.666667,17296.666667,8495.000000,10873.333333,9668.333333,1205.0,10873.333333,26126.666667,22290.666667,1957.333333,1245.666667,550.333333,717.000000,1689.666667,5426.000000,2723.666667,4271.333333,1632.333333,26406.333333,25672.000000,18006.666667,754

In [31]:
city_census_cleaned.describe()

,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
count,2469.000000,2469.000000,2.469000e+03,2.469000e+03,2.469000e+03,2.469000e+03,2469.000000,2.469000e+03,2469.000000,2.469000e+03,2469.000000,2.469000e+03,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2.469000e+03,2469.000000,2469.000000,2469.000000,2.469000e+03,2469.000000,2.469000e+03,2469.000000,2469.000000,2.469000e+03,2.469000e+03,2.469000e+03,2.469000e+03,2469.000000,2469.000000,2.469000e+03,2.469000e+03,2.469000e+03,2.469000e+03,2.469000e+03,2469.000000,2.469000e+03,2.469000e+03,2.469000e+03,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2.469000e+03,2469.000000,2.469000e+03,2469.000000,2.469000e+03,2.469000e+03,2.469000e+03,2469.000000,2469.000000
mean,1248.818145,37.397286,3.592967e+04,3.751762e+04,5.819720e+04,1.504537e+04,635.493771,7.748256e+03,232.505600,7.120515e+03,84887.239875,2.707206e+04,70915.293087,4240.219111,4745.952559,4149.298666,5911.780948,3990.245950,1834.774391,2199.789774,1.039932e+04,3237.371954,6056.853412,1105.092264,6.142798e+04,9059.346905,3.369511e+04,3558.229194,16980.418579,9.949962e+03,3.206503e+03,3.738585e+04,2.704639e+04,2071.666830,1991.359802,7.160715e+04,4.128460e+04,3.032256e+04,2.995741e+04,2.704283e+04,2914.578811,3.456223e+04,8.088718e+04,6.790860e+04,7561.403369,2592.251756,2143.653551,2926.810940,3342.706824,1.182081e+04,9809.977807,1.067517e+04,6782.200115,7.350519e+04,5.974636e+04,3.925302e+04,19034.788986,1458.552502
std,723.493867,5.796310,1.106134e+05,1.178289e+05,1.123406e+05,5.509530e+04,1736.925331,3.022793e+04,763.794625,3.367562e+04,34276.683363,8.514589e+04,29925.956408,16101.572502,14558.773069,11976.985368,17049.973923,12001.382218,6085.464511,9067.951277,3.969632e+04,12222.591336,22582.791514,5227.884749,1.853887e+05,25378.342783,1.052637e+05,6129.752755,39619.313859,4.284967e+04,2.282900e+04,1.105805e+05,5.547398e+04,12157.609477,5782.326955,2.234443e+05,9.404886e+04,1.330923e+05,9.457646e+04,8.514933e+04,9860.559571,9.402730e+04,2.244412e+05,1.970055e+05,18393.068911,4841.239040,4557.413424,15107.343932,13317.117759,3.735576e+04,25436.301431,3.590045e+04,24649.731381,2.283031e+05,1.580315e+05,1.149644e+05,42195.687268,7012.269305
min,1.000000,13.750000,7.861000e+03,6.283667e+03,1.534000e+03,5.800000e+01,0.000000,0.000000e+00,0.000000,0.000000e+00,19782.500000,2.755667e+03,12581.750000,0.000000,0.000000,116.000000,313.000000,0.000000,0.000000,0.000000,1.960000e+02,0.000000,0.000000,0.000000,5.281000e+03,29.000000,2.327000e+03,0.000000,607.333333,1.990000e+02,0.000000e+00,2.827333e+03,1.570000e+03,0.000000,0.000000,7.590000e+03,1.560000e+02,1.370000e+02,3.887333e+03,2.755667e+03,0.000000,4.020333e+03,1.881600e+04,6.457667e+03,89.000000,0.000000,0.00000

In [32]:
#default values
median_age = 37.3
male = 35929
female = 37517
white_alone = 58197
black_africa = 15045
native = 635
asian = 7748
islander = 232
other = 7120
median_family_income = 84887
household_income = 27072
median_household_income = 70915
household_less_20 = 4240
household_20_39 = 4745
household_40_59 = 4149
household_60_99 = 5911
household_100_149 = 3990
household_150_200 = 1834
household_200_more = 2199
below_poverty = 10399
below_poverty_under_18 = 3237
below_poverty_18_64 = 6056
below_poverty_65_over = 1105
above_poverty = 61427
above_poverty_65_over = 9059
travel_time = 33695
travel_time_less_10 = 3558
travel_time_10_29 = 16980
travel_time_30_59 = 9949
travel_time_60_more = 3206
means_transit = 37385
means_transit_drove_alone = 27046
means_transit_other = 2071
means_transit_worked_home = 1991
total_pop_in_occupied_housing = 71607
owner_occupied = 41284
renter_occupied = 30322
total_occupancy = 29957
occupied = 27042
vacant = 2914
total_housing_units = 34562
total_geographical_mobility = 80887
same_house_1_year = 67908
moved_within_same_county = 7561
moved_from_diff_county_within_same_state = 2592
moved_from_diff_state = 2143
edu_less_9 = 2926
edu_9_12 = 3342
edu_high_school = 11820
edu_no_degree = 9809
edu_bachelor_degree = 10675
edu_professional_degree = 6782
total_birth_in_us = 73505
birth_native = 59746
birth_residence_state = 39253
birth_other_state = 19034
birth_outside_us = 1458

Selecting the variables

In [33]:
x = city_census_cleaned.loc[:,['median_age', 'male', 'female', 'white_alone',
       'black_africa', 'native', 'asian', 'islander', 'other',
       'median_family_income', 'household_income', 'median_household_income',
       'household_less_20', 'household_20_39', 'household_40_59',
       'household_60_99', 'household_100_149', 'household_150_200',
       'household_200_more', 'below_poverty', 'below_poverty_under_18',
       'below_poverty_18_64', 'below_poverty_65_over', 'above_poverty',
       'above_poverty_65_over', 'travel_time', 'travel_time_less_10',
       'travel_time_10_29', 'travel_time_30_59', 'travel_time_60_more',
       'means_transit', 'means_transit_drove_alone', 'means_transit_other',
       'means_transit_worked_home', 'total_pop_in_occupied_housing',
       'owner_occupied', 'renter_occupied', 'total_occupancy', 'occupied',
       'vacant', 'total_housing_units', 'total_geographical_mobility',
       'same_house_1_year', 'moved_within_same_county',
       'moved_from_diff_county_within_same_state', 'moved_from_diff_state',
       'edu_less_9', 'edu_9_12', 'edu_high_school', 'edu_no_degree',
       'edu_bachelor_degree', 'edu_professional_degree', 'total_birth_in_us',
       'birth_native', 'birth_residence_state', 'birth_other_state',
       'birth_outside_us']].values

In [34]:
nbrs = NearestNeighbors(n_neighbors=5).fit(x)

10% to female,30% to white people, 20% to above poverty, 10% to travel time less than 10, 20% to education high school, 10% to householf income 60 to 99.

In [35]:
female = female + ((10*female)/100)
white_alone = white_alone + ((30*white_alone)/100)
above_poverty = above_poverty + ((20*above_poverty)/100)
travel_time_less_10 = travel_time_less_10 + ((10 * travel_time_less_10)/100)
edu_high_school = edu_high_school + ((20*edu_high_school)/100)
household_60_99 = household_60_99 + ((10 * household_60_99)/100)

In [36]:
female

41268.7

In [37]:
inp = [median_age, male, female, white_alone,
       black_africa, native, asian, islander, other,
       median_family_income, household_income, median_household_income,
       household_less_20, household_20_39, household_40_59,
       household_60_99, household_100_149, household_150_200,
       household_200_more, below_poverty, below_poverty_under_18,
       below_poverty_18_64, below_poverty_65_over, above_poverty,
       above_poverty_65_over, travel_time, travel_time_less_10,
       travel_time_10_29, travel_time_30_59, travel_time_60_more,
       means_transit, means_transit_drove_alone, means_transit_other,
       means_transit_worked_home, total_pop_in_occupied_housing,
       owner_occupied, renter_occupied, total_occupancy, occupied,
       vacant, total_housing_units, total_geographical_mobility,
       same_house_1_year, moved_within_same_county,
       moved_from_diff_county_within_same_state, moved_from_diff_state,
       edu_less_9, edu_9_12, edu_high_school, edu_no_degree,
       edu_bachelor_degree, edu_professional_degree, total_birth_in_us,
       birth_native, birth_residence_state, birth_other_state,
       birth_outside_us]
inp

[37.3,
 35929,
 41268.7,
 75656.1,
 15045,
 635,
 7748,
 232,
 7120,
 84887,
 27072,
 70915,
 4240,
 4745,
 4149,
 6502.1,
 3990,
 1834,
 2199,
 10399,
 3237,
 6056,
 1105,
 73712.4,
 9059,
 33695,
 3913.8,
 16980,
 9949,
 3206,
 37385,
 27046,
 2071,
 1991,
 71607,
 41284,
 30322,
 29957,
 27042,
 2914,
 34562,
 80887,
 67908,
 7561,
 2592,
 2143,
 2926,
 3342,
 14184.0,
 9809,
 10675,
 6782,
 73505,
 59746,
 39253,
 19034,
 1458]

In [38]:
nbrs.kneighbors([inp])

(array([[33382.45094017, 34097.90297834, 34116.18125445, 34336.50575906,
         34598.94015422]]), array([[1340, 1239, 1486,  733, 1590]]))

In [39]:
city_census_cleaned.iloc[[1340,1239,1486,733,1590]]

,city_id,median_age,male,female,white_alone,black_africa,native,asian,islander,other,median_family_income,household_income,median_household_income,household_less_20,household_20_39,household_40_59,household_60_99,household_100_149,household_150_200,household_200_more,below_poverty,below_poverty_under_18,below_poverty_18_64,below_poverty_65_over,above_poverty,above_poverty_65_over,travel_time,travel_time_less_10,travel_time_10_29,travel_time_30_59,travel_time_60_more,means_transit,means_transit_drove_alone,means_transit_other,means_transit_worked_home,total_pop_in_occupied_housing,owner_occupied,renter_occupied,total_occupancy,occupied,vacant,total_housing_units,total_geographical_mobility,same_house_1_year,moved_within_same_county,moved_from_diff_county_within_same_state,moved_from_diff_state,edu_less_9,edu_9_12,edu_high_school,edu_no_degree,edu_bachelor_degree,edu_professional_degree,total_birth_in_us,birth_native,birth_residence_state,birth_other_state,birth_outside_us
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
1340,1353,39.025000,33405.250000,37031.500000,57556.250000,3935.000000,665.000000,2552.000000,39.000000,3093.750000,85107.750000,26991.250000,68204.25,2488.750000,4638.500000,4745.250000,6575.000000,4368.500000,2191.750000,1983.500000,5321.500000,1955.500000,2879.000000,487.000000,64311.000000,10725.250000,33740.500000,3357.500000,15812.750000,12655.750000,1914.500000,36186.000000,29860.750000,517.250000,2445.500000,70098.000000,46559.000000,23539.000000,28301.500000,26991.250000,1310.250000,28301.500000,69693.000000,57555.000000,8112.250000,2372.250000,1125.000000,1032.000000,2569.500000,11376.500000,12491.250000,12252.500000,4699.250000,70436.750000,64594.500000,39776.250000,23407.250000,1411.000000
1239,1252,35.533333,38064.333333,39675.333333,60298.333333,8013.333333,273.333333,6578.333333,30.666667,585.000000,91890.000000,31719.666667,67508.00,5865.333333,4338.000000,4041.666667,7564.333333,5289.666667,2170.000000,2450.666667,9990.666667,1830.333333,6776.000000,1384.333333,65520.000000,8568.333333,36836.666667,9410.000000,22795.333333,3637.333333,994.000000,38605.000000,31667.000000,1832.333333,1768.333333,75577.333333,49326.333333,26251.000000,35037.333333,31719.666667,3317.666667,35037.333333,76991.000000,66529.666667,6317.000000,3053.000000,690.666667,1286.000000,1682.000000,10830.000000,9488.000000,16097.333333,8531.333333,77739.666667,69373.333333,52988.000000,15700.333333,685.000000
1486,1504,35.733333,39667.333333,41806.666667,49825.666667,5514.333333,1608.333333,9732.333333,1503.000000,6655.666667,87626.333333,30469.000000,70017.00,3429.000000,4788.000000,4540.333333,7331.000000,5913.333333,2639.666667,1827.666667,9728.666667,3729.333333,5343.333333,656.000000,71089.666667,9161.666667,38291.666667,2765.666667,15480.666667,13438.000000,6607.333333,40069.000000,30128.000000,879.000000,1777.333333,80887.333333,46530.666667,34356.666667,32478.000000,30469.000000,2009.000000,32478.000000,80401.000000,65635.666667,8555.333333,3650.333333,1580.333333,2492.333333,3097.333333,15973.333333,12936.000000,10477.333333,4433.666667,81474.000000,62912.333333,37409.333333,24266.333333,1236.666667
733,740,39.533333,38894.333333,40163.333333,64042.666667,541.333333,476.666667,2029.666667,61.666667,8911.666667,98527.666667,28618.000000,84957.00,2166.333333,3793.333333,4115.666667,6576.000000,5402.666667,3182.333333,3381.666667,5453.000000,1243.666667,3311.666667,897.666667,73014.000000,13034.333333,37643.333333,6396.000000,20026.000000,7652.000000,3569.333333,39895.666667,31343.666667,2295.666667,2252.333333,78148.333333,46533.000000,31615.333333,31400.666667,28618.000000,2782.666667,31400.666667,78462.666667,69535.333333,5260.333333,2654.333333,603.666667,5334.000000,2730.333333,10373

In [40]:
city[(city['ID']==1353) | (city['ID']==1252) | (city['ID']==1504) | (city['ID']==740) | (city['ID']==1608)]

,ID,city,state,state_code,code,time_created
739,740,Napa city,California,6,50258,2021-01-19 14:47:03
1251,1252,Bloomington city,Illinois,17,6613,2021-01-19 14:47:03
1352,1353,North Richland Hills city,Texas,48,52356,2021-01-19 14:47:03
1503,1504,Auburn city,Washington,53,3180,2021-01-19 14:47:03
1607,1608,Waukesha city,Wisconsin,55,84250,2021-01-19 14:47:03
